In [2]:
import pandas as pd
from utils import create_missing_data
from utils.load_dataset import load_eval_columns

from dataset_config import MIMICIV_config

In [3]:
from imputers.knn import KNN as knn
from imputers.mice import MiceImputer as mice
from imputers.nnimputer import NeuralNetworkImputer
from imputers.ssl_imputer import SSLImputer
from evals.eval import Evaluation

In [20]:
data = pd.read_csv("F:\\Jobs\\InfAI\\INTELLILUNG_2-main\\INTELLILUNG_2-main\\MIMIC\\data\\mimiciv_state_vectors.csv")

In [3]:
# Dropping columns with 50% missing values
col_to_drop = ['ecmo_sweep', 'vital_co', 'is_controlled', 'vent_rsbi', 'blood_ffp', 'blood_prbc', 
                'blood_album', 'ecmo_rpm', 'ecmo_bloodflow', 'blood_sco2', 'vital_SVRI', 'blood_svo2',
                    'vent_etco2', 'vital_mpap', 'blood_smvo2', 'vital_cvp']
data = data.drop(columns=col_to_drop)
data.columns

Index(['stay_id', 'mv_id', 'timepoints', 'age', 'blood_be', 'blood_hco3',
       'blood_ph', 'drugs_vaso4h', 'vital_map', 'vital_DBP', 'vital_SBP',
       'blood_INR', 'blood_PTT', 'daemo_sex', 'daemo_weight', 'daemo_height',
       'daemo_discharge', 'ecmo_active', 'blood_calcium', 'blood_chlorid',
       'blood_caion', 'blood_magnes', 'blood_potas', 'blood_sodium',
       'cum_fluid_balance', 'state_ivfluid4h', 'blood_paco2', 'blood_pao2',
       'vent_fio2', 'vital_spo2', 'blood_sao2', 'blood_plat', 'blood_hb',
       'blood_hct', 'blood_wbc', 'vital_hr', 'state_bun', 'blood_crea',
       'state_urin4h', 'blood_ast', 'blood_alt', 'blood_billi', 'blood_lac',
       'blood_gluco', 'state_temp', 'vent_inspexp', 'vent_insp', 'vent_pinsp',
       'vent_mairpress', 'vent_mv', 'vent_peep', 'vent_rrtot',
       'vent_rrcontrolled', 'vent_rrspont', 'vent_suppress', 'vent_vt',
       'vent_vtnorm', 'vent_mode', 'state_airtype', 'daemo_morta',
       'episode_id'],
      dtype='object')

In [5]:
state_vectors = data.iloc[:, 4:-1] # Dropping the stayid, mv_id, charttime, age and episode_id columns
state_vectors.to_csv('mimiciv_missing_data.csv', index=False)
state_vectors.shape

(480060, 56)

In [6]:
state_vectors = state_vectors.dropna()
state_vectors.to_csv('mimiciv_full_data.csv', index=False)
state_vectors.shape


(138021, 56)

In [4]:
data_cfg = MIMICIV_config()

mimic_data = pd.read_csv(data_cfg.full_data_path)
mimic_data.shape

(138021, 56)

In [5]:
data_cfg.state_vector = mimic_data.columns
data_cfg.state_vector

Index(['blood_be', 'blood_hco3', 'blood_ph', 'drugs_vaso4h', 'vital_map',
       'vital_DBP', 'vital_SBP', 'blood_INR', 'blood_PTT', 'daemo_sex',
       'daemo_weight', 'daemo_height', 'daemo_discharge', 'ecmo_active',
       'blood_calcium', 'blood_chlorid', 'blood_caion', 'blood_magnes',
       'blood_potas', 'blood_sodium', 'cum_fluid_balance', 'state_ivfluid4h',
       'blood_paco2', 'blood_pao2', 'vent_fio2', 'vital_spo2', 'blood_sao2',
       'blood_plat', 'blood_hb', 'blood_hct', 'blood_wbc', 'vital_hr',
       'state_bun', 'blood_crea', 'state_urin4h', 'blood_ast', 'blood_alt',
       'blood_billi', 'blood_lac', 'blood_gluco', 'state_temp', 'vent_inspexp',
       'vent_insp', 'vent_pinsp', 'vent_mairpress', 'vent_mv', 'vent_peep',
       'vent_rrtot', 'vent_rrcontrolled', 'vent_rrspont', 'vent_suppress',
       'vent_vt', 'vent_vtnorm', 'vent_mode', 'state_airtype', 'daemo_morta'],
      dtype='object')

In [5]:
column_index = mimic_data.columns.get_loc('blood_chlorid')
print(column_index)


15


### Create Missing dataset
##### Params: 
#####         dataset name, 
#####         dataset/dataframe, 
#####         columns in which we need to add missing values, 
#####         stepped missingness, 
#####         random missing, 
#####         percent missing

In [6]:
create_missing_data.create_missing_dataset("mimiciv_blood_chlorid", mimic_data, mimic_data.columns[15:16], steps=None, random=True, percent=30)

Missing Data saved in "Dataset" Folder


In [5]:
data_cfg.missing_data_path

'dataset/mimiciv_blood_chlorid_w_missing_values_random.csv'

In [4]:
test_data = pd.read_csv(data_cfg.missing_data_path)
test_data.shape

(138021, 56)

##### For data having missing values in more than one columns and you want to use the GRU based imputation, please refer the main.py

In [6]:
#knn.impute_data(test_data, data_cfg)
mice.mice_impute(test_data, data_cfg)

Imputing: 100%|██████████| 138021/138021 [00:23<00:00, 5838.49rows/s]


Saving the imputed data 

Imputed Data saved at location "dataset/mimiciv_blood_chlorid_imp_mice.csv"


In [7]:
mice.mice_forest(test_data, data_cfg)

mf
Saving the imputed data 

Imputed Data saved at location "dataset/mimiciv_blood_chlorid_imp_miceforest.csv"


In [7]:
gru = NeuralNetworkImputer(data_cfg)
missing_data = pd.read_csv(data_cfg.missing_data_path)
for col in data_cfg.missing_state_vector:
    col_imputations = gru.fit_transform(col)
    missing_data = gru.fill_imputation(missing_data, col, col_imputations)

gru.save_imputed_data(data_cfg.imputed_data_path, missing_data)


Training Started for blood_chlorid

Epoch 1/50
2709/2718 [============================>.] - ETA: 0s - loss: 164.5344 - root_mean_squared_error: 12.8271 - r2_score: -2.7519INFO:tensorflow:Assets written to: nnModels/MIMICIV_model_300424_161817/col_blood_chlorid\assets


INFO:tensorflow:Assets written to: nnModels/MIMICIV_model_300424_161817/col_blood_chlorid\assets


2718/2718 [==============================] - 9s 3ms/step - loss: 164.0522 - root_mean_squared_error: 12.8083 - r2_score: -2.7394 - val_loss: 5.8765 - val_root_mean_squared_error: 2.4241 - val_r2_score: 0.8664
Epoch 2/50
2718/2718 [==============================] - 5s 2ms/step - loss: 5.8329 - root_mean_squared_error: 2.4151 - r2_score: 0.8670 - val_loss: 6.3482 - val_root_mean_squared_error: 2.5196 - val_r2_score: 0.8557
Epoch 3/50
2699/2718 [============================>.] - ETA: 0s - loss: 5.1392 - root_mean_squared_error: 2.2670 - r2_score: 0.8829INFO:tensorflow:Assets written to: nnModels/MIMICIV_model_300424_161817/col_blood_chlorid\assets


INFO:tensorflow:Assets written to: nnModels/MIMICIV_model_300424_161817/col_blood_chlorid\assets


2718/2718 [==============================] - 7s 2ms/step - loss: 5.1386 - root_mean_squared_error: 2.2669 - r2_score: 0.8829 - val_loss: 4.7551 - val_root_mean_squared_error: 2.1806 - val_r2_score: 0.8919
Epoch 4/50
2718/2718 [==============================] - 5s 2ms/step - loss: 4.5201 - root_mean_squared_error: 2.1261 - r2_score: 0.8969 - val_loss: 5.7940 - val_root_mean_squared_error: 2.4071 - val_r2_score: 0.8683
Epoch 5/50
2699/2718 [============================>.] - ETA: 0s - loss: 4.2761 - root_mean_squared_error: 2.0679 - r2_score: 0.9026INFO:tensorflow:Assets written to: nnModels/MIMICIV_model_300424_161817/col_blood_chlorid\assets


...

2718/2718 [==============================] - 7s 2ms/step - loss: 2.1222 - root_mean_squared_error: 1.4568 - r2_score: 0.9516 - val_loss: 2.4079 - val_root_mean_squared_error: 1.5518 - val_r2_score: 0.9453
Epoch 37/50
2718/2718 [==============================] - 5s 2ms/step - loss: 2.2185 - root_mean_squared_error: 1.4895 - r2_score: 0.9494 - val_loss: 3.0158 - val_root_mean_squared_error: 1.7366 - val_r2_score: 0.9314
Epoch 38/50
2718/2718 [==============================] - 5s 2ms/step - loss: 2.1260 - root_mean_squared_error: 1.4581 - r2_score: 0.9515 - val_loss: 2.4830 - val_root_mean_squared_error: 1.5757 - val_r2_score: 0.9436
Epoch 39/50
2718/2718 [==============================] - 5s 2ms/step - loss: 2.1323 - root_mean_squared_error: 1.4602 - r2_score: 0.9514 - val_loss: 2.4381 - val_root_mean_squared_error: 1.5615 - val_r2_score: 0.9446
Epoch 40/50
2718/2718 [==============================] - 5s 2ms/step - loss: 2.0843 - root_mean_squared_error: 1.4437 - r2_score: 0.9525 - val_l

INFO:tensorflow:Assets written to: nnModels/MIMICIV_model_300424_161817/col_blood_chlorid\assets


2718/2718 [==============================] - 6s 2ms/step - loss: 2.0353 - root_mean_squared_error: 1.4267 - r2_score: 0.9536 - val_loss: 2.3689 - val_root_mean_squared_error: 1.5391 - val_r2_score: 0.9461
Epoch 46/50
2706/2718 [============================>.] - ETA: 0s - loss: 2.0188 - root_mean_squared_error: 1.4209 - r2_score: 0.9540INFO:tensorflow:Assets written to: nnModels/MIMICIV_model_300424_161817/col_blood_chlorid\assets


INFO:tensorflow:Assets written to: nnModels/MIMICIV_model_300424_161817/col_blood_chlorid\assets


2718/2718 [==============================] - 7s 2ms/step - loss: 2.0189 - root_mean_squared_error: 1.4209 - r2_score: 0.9540 - val_loss: 2.3151 - val_root_mean_squared_error: 1.5216 - val_r2_score: 0.9474
Epoch 47/50
2718/2718 [==============================] - 5s 2ms/step - loss: 2.1173 - root_mean_squared_error: 1.4551 - r2_score: 0.9517 - val_loss: 2.3771 - val_root_mean_squared_error: 1.5418 - val_r2_score: 0.9460
Epoch 48/50
2718/2718 [==============================] - 5s 2ms/step - loss: 2.1076 - root_mean_squared_error: 1.4518 - r2_score: 0.9519 - val_loss: 2.3599 - val_root_mean_squared_error: 1.5362 - val_r2_score: 0.9463
Epoch 49/50
2718/2718 [==============================] - 5s 2ms/step - loss: 2.0830 - root_mean_squared_error: 1.4432 - r2_score: 0.9525 - val_loss: 2.3438 - val_root_mean_squared_error: 1.5309 - val_r2_score: 0.9467
Epoch 50/50
2718/2718 [==============================] - 5s 2ms/step - loss: 2.1088 - root_mean_squared_error: 1.4522 - r2_score: 0.9519 - val_l

In [8]:
ssl_imputer = SSLImputer(data_cfg)
ssl_imputer.build_model()
ssl_imputer.fit(mimic_data) # Need to fit on not null data or full data.
ssl_imputer.transform(test_data)


Training Started...

Epoch 1/50

3416/3451 [============================>.] - ETA: 0s - loss: 0.7685 - root_mean_squared_error: 0.8767 - r2_score: 0.2314INFO:tensorflow:Assets written to: AEModels/MIMICIV_model_240424_145624\assets


INFO:tensorflow:Assets written to: AEModels/MIMICIV_model_240424_145624\assets


3451/3451 [==============================] - 6s 2ms/step - loss: 0.7668 - root_mean_squared_error: 0.8757 - r2_score: 0.2332 - val_loss: 0.5742 - val_root_mean_squared_error: 0.7578 - val_r2_score: 0.4110
Epoch 2/50
3427/3451 [============================>.] - ETA: 0s - loss: 0.4889 - root_mean_squared_error: 0.6992 - r2_score: 0.5114INFO:tensorflow:Assets written to: AEModels/MIMICIV_model_240424_145624\assets


INFO:tensorflow:Assets written to: AEModels/MIMICIV_model_240424_145624\assets


3451/3451 [==============================] - 5s 1ms/step - loss: 0.4881 - root_mean_squared_error: 0.6987 - r2_score: 0.5119 - val_loss: 0.4024 - val_root_mean_squared_error: 0.6343 - val_r2_score: 0.5850
Epoch 3/50
3404/3451 [============================>.] - ETA: 0s - loss: 0.3687 - root_mean_squared_error: 0.6072 - r2_score: 0.6284INFO:tensorflow:Assets written to: AEModels/MIMICIV_model_240424_145624\assets


...

3451/3451 [==============================] - 5s 1ms/step - loss: 0.0366 - root_mean_squared_error: 0.1913 - r2_score: 0.9634 - val_loss: 0.0230 - val_root_mean_squared_error: 0.1515 - val_r2_score: 0.9678
Epoch 49/50
3449/3451 [============================>.] - ETA: 0s - loss: 0.0355 - root_mean_squared_error: 0.1883 - r2_score: 0.9645INFO:tensorflow:Assets written to: AEModels/MIMICIV_model_240424_145624\assets


INFO:tensorflow:Assets written to: AEModels/MIMICIV_model_240424_145624\assets


3451/3451 [==============================] - 5s 1ms/step - loss: 0.0355 - root_mean_squared_error: 0.1883 - r2_score: 0.9645 - val_loss: 0.0220 - val_root_mean_squared_error: 0.1482 - val_r2_score: 0.9689
Epoch 50/50
3415/3451 [============================>.] - ETA: 0s - loss: 0.0346 - root_mean_squared_error: 0.1859 - r2_score: 0.9656INFO:tensorflow:Assets written to: AEModels/MIMICIV_model_240424_145624\assets


INFO:tensorflow:Assets written to: AEModels/MIMICIV_model_240424_145624\assets


3451/3451 [==============================] - 5s 1ms/step - loss: 0.0343 - root_mean_squared_error: 0.1853 - r2_score: 0.9657 - val_loss: 0.0209 - val_root_mean_squared_error: 0.1447 - val_r2_score: 0.9699
Training Completed...

Imputation Started...

4314/4314 [==============================] - 3s 681us/step
Imputation Completed...

Saving the imputed data 

Imputed Data saved at location "dataset/mimiciv_blood_chlorid_imp_ssl.csv"


In [9]:
cols = mimic_data.columns[15:16]
cols

Index(['blood_chlorid'], dtype='object')

In [12]:
mice_imp = pd.read_csv('dataset/mimiciv_blood_chlorid_imp_mice.csv', usecols=cols)
ssl_imp = pd.read_csv('dataset/mimiciv_blood_chlorid_imp_ssl.csv', usecols=cols)
mf_imp = pd.read_csv('dataset/mimiciv_blood_chlorid_imp_miceforest.csv', usecols=cols)
mice_imp.shape, ssl_imp.shape

((138021, 1), (138021, 1))

In [13]:
import numpy as np
from sklearn.metrics import mean_squared_error

def normalize_mse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    normalized_mse = mse / (np.max(y_true) - np.min(y_true))**2
    return normalized_mse

for col in cols:
    mask = test_data[col].isna()
    og_col = mimic_data[mask][col]
    mice_col = mice_imp[mask][col]
    ssl_col = ssl_imp[mask][col]
    mf_col = mf_imp[mask][col]

    mse_mice = normalize_mse(og_col, mice_col)
    mse_ssl = normalize_mse(og_col, ssl_col)
    mse_mf = normalize_mse(og_col, mf_col)

    print(f"{col} ", end="")
    print(f"MICE: {round(mse_mice, 4)} | ", end="")
    print(f"SSL: {round(mse_ssl, 4)} | ", end="")
    print(f"MICEForest: {round(mse_mf, 4)}")
    print()


blood_chlorid MICE: 0.0018 | SSL: 0.0056 | MICEForest: 0.0011



In [8]:
imputers = ["gru"]    # list the imputers for which yoiu want to evaluate the imputation
eval = Evaluation()

In [9]:
for imputer in imputers:
    eval_cols_dict = load_eval_columns(data_cfg, imputer)
    for i, col in enumerate(data_cfg.missing_state_vector, start=1):
        og_col = eval_cols_dict[f"og_{col}_{i}"]
        imputed_col = eval_cols_dict[f"imputed_{col}_{i}"]
        eval.scores(og_col, imputed_col, data_cfg.imputed_data_path, imputer, col)

Evalution of blood_chlorid for gru saved at evals/evaluations.csv.


In [28]:
milos_mimic = pd.read_csv("C:\\Users\\prath\\Downloads\\mimiciv_state_vectors_2024_april_16_vent_pinsp-peep.csv", usecols=['cum_fluid_balance', 'state_ivfluid4h'])
milos_mimic.describe()

,cum_fluid_balance,state_ivfluid4h
count,482467.000000,482467.000000
mean,284.129112,296.171967
std,361.821930,348.490029
min,-3500.000000,0.000000
25%,64.994556,70.002670
50%,195.016680,200.000000
75%,400.211955,402.010059
max,12886.321896,12886.321896
